In [1]:
import pygame

class Board:
    def __init__(self, window_height, window_width, header_height=50, palette_height=50, reset_button_height=30, difficulty_button_height=30):
        self.header_height = header_height
        self.palette_height = palette_height
        self.reset_button_height = reset_button_height
        self.difficulty_button_height = difficulty_button_height
        self.board_height = window_height - palette_height - header_height
        self.width = window_width
        self.colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 165, 0), (128, 0, 128)]  # Red, Green, Blue, Yellow, Orange, Purple
        self.text_colors = [(0, 0, 0), (255, 255, 255), (255, 255, 255), (0, 0, 0), (0, 0, 0), (255, 255, 255)]  # Contrasting text colors
        self.reset_button_width = 80
        self.difficulty_button_width = 150
        self.sort_button_width = 80
        pygame.display.set_caption('Color Change Game with Palette')
        self.font = pygame.font.SysFont(None, 24)  # Reduced font size
        self.screen = pygame.display.set_mode((self.width, window_height))

    def draw_board(self, state):
        cell_width = self.width // int(state.n**0.5)
        cell_height = self.board_height // int(state.n**0.5)
        for index in range(len(state.cells)):
            y, x = divmod(index, int(state.n**0.5))
            if state.cells[index] != -1:  # Only draw cells with valid colors
                color = self.colors[state.cells[index]]
                text_color = self.text_colors[state.cells[index]]
                pygame.draw.rect(self.screen, color,
                                 (x * cell_width, y * cell_height + self.header_height, cell_width, cell_height))
                text = self.font.render(str(state.clicks[index]), True, text_color)
                text_rect = text.get_rect(center=(x * cell_width + cell_width // 2,
                                                  y * cell_height + self.header_height + cell_height // 2))
                self.screen.blit(text, text_rect)

    def draw_score(self, message):
        score_text = self.font.render(message, True, (255, 255, 255))
        score_rect = score_text.get_rect(center=(self.width // 2, self.header_height // 2))
        self.screen.blit(score_text, score_rect)

    def draw_palette(self, state, selected_color=None):
        palette_y = self.board_height + self.header_height  # Position the palette below the board
        color_counts = state.get_palette_color_count()
        for i in range(state.ncolors):
            color = self.colors[i]
            pygame.draw.rect(self.screen, color,
                             (i * (self.width // state.ncolors), palette_y, self.width // state.ncolors, self.palette_height))
            if i == selected_color:
                pygame.draw.rect(self.screen, (0, 0, 0),
                                 (i * (self.width // state.ncolors), palette_y, self.width // state.ncolors, self.palette_height), 3)
            count_text = self.font.render(str(color_counts[i]), True, (0, 0, 0))
            count_rect = count_text.get_rect(center=(i * (self.width // state.ncolors) + (self.width // state.ncolors) // 2, palette_y + self.palette_height // 2))
            self.screen.blit(count_text, count_rect)

    def draw_reset_button(self):
        reset_button_x = self.width - self.reset_button_width - 10
        reset_button_y = 10
        pygame.draw.rect(self.screen, (200, 200, 200),
                         (reset_button_x, reset_button_y, self.reset_button_width, self.reset_button_height))
        reset_text = self.font.render('Reset', True, (0, 0, 0))
        reset_rect = reset_text.get_rect(center=(reset_button_x + self.reset_button_width // 2,
                                                 reset_button_y + self.reset_button_height // 2))
        self.screen.blit(reset_text, reset_rect)

    def draw_difficulty_button(self, current_difficulty):
        difficulty_button_x = 10
        difficulty_button_y = 10
        pygame.draw.rect(self.screen, (200, 200, 200),
                         (difficulty_button_x, difficulty_button_y, self.difficulty_button_width, self.difficulty_button_height))
        difficulty_text = self.font.render(f'Difficulty: {current_difficulty}', True, (0, 0, 0))
        difficulty_rect = difficulty_text.get_rect(center=(difficulty_button_x + self.difficulty_button_width // 2,
                                                           difficulty_button_y + self.difficulty_button_height // 2))
        self.screen.blit(difficulty_text, difficulty_rect)

    def draw_sort_button(self):
        sort_button_x = 10 + self.difficulty_button_width + 10
        sort_button_y = 10
        pygame.draw.rect(self.screen, (200, 200, 200),
                         (sort_button_x, sort_button_y, self.sort_button_width, self.reset_button_height))
        sort_text = self.font.render('Sort', True, (0, 0, 0))
        sort_rect = sort_text.get_rect(center=(sort_button_x + self.sort_button_width // 2,
                                               sort_button_y + self.reset_button_height // 2))
        self.screen.blit(sort_text, sort_rect)

    def draw_highest_score(self, highest_score):
        highest_score_x = self.width - self.reset_button_width - self.sort_button_width - 30
        highest_score_y = 10
        highest_score_text = self.font.render(f'High Score: {highest_score}', True, (255, 255, 255))
        highest_score_rect = highest_score_text.get_rect(center=(highest_score_x + self.sort_button_width // 2,
                                                                 highest_score_y + self.reset_button_height // 2))
        self.screen.blit(highest_score_text, highest_score_rect)

    def draw_all(self, state, message, selected_color, current_difficulty, highest_score):
        self.screen.fill((0, 0, 0))
        self.draw_board(state)
        self.draw_score(message)
        self.draw_palette(state, selected_color)
        self.draw_reset_button()
        self.draw_difficulty_button(current_difficulty)
        self.draw_sort_button()
        self.draw_highest_score(highest_score)
        pygame.display.flip()

    def get_clicked_cell(self, pos, size):
        cell_width = self.width // size
        cell_height = self.board_height // size
        x = pos[0] // cell_width
        y = (pos[1] - self.header_height) // cell_height
        return y * size + x  # Convert 2D position to 1D index

    def is_click_on_palette(self, pos):
        palette_y = self.board_height + self.header_height
        return palette_y <= pos[1] <= palette_y + self.palette_height

    def get_palette_color(self, pos, ncolors):
        return pos[0] // (self.width // ncolors)

    def is_click_on_reset_button(self, pos):
        reset_button_x = self.width - self.reset_button_width - 10
        reset_button_y = 10
        return reset_button_x <= pos[0] <= reset_button_x + self.reset_button_width and \
               reset_button_y <= pos[1] <= reset_button_y + self.reset_button_height

    def is_click_on_difficulty_button(self, pos):
        difficulty_button_x = 10
        difficulty_button_y = 10
        return difficulty_button_x <= pos[0] <= difficulty_button_x + self.difficulty_button_width and \
               difficulty_button_y <= pos[1] <= difficulty_button_y + self.difficulty_button_height

    def is_click_on_sort_button(self, pos):
        sort_button_x = 10 + self.difficulty_button_width + 10
        sort_button_y = 10
        return sort_button_x <= pos[0] <= sort_button_x + self.sort_button_width and \
               sort_button_y <= pos[1] <= sort_button_y + self.reset_button_height


pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import pygame
from state import State

class Game:
    def __init__(self):
        pygame.init()
        self.window_height = 800  # Increased window height to accommodate the palette
        self.window_width = 800
        self.board = Board(self.window_height, self.window_width)
        self.current_difficulty = 'easy'
        self.state_levels = {
            'easy': State(n=3 * 3, ncolors=4, limit=3),
            'medium': State(n=5 * 5, ncolors=5, limit=4),
            'hard': State(n=7 * 7, ncolors=6, limit=4)
        }
        self.state_levels['medium'].reset()
        self.state_levels['hard'].reset()
        self.state = self.state_levels[self.current_difficulty]
        self.initial_message = 'Please select a color from the palette'
        self.show_initial_message = True
        self.first_click = False
        self.selected_color = 0  # Default selected color
        self.highest_scores = {'easy': 0, 'medium': 0, 'hard': 0}

    def set_difficulty(self, difficulty):
        self.current_difficulty = difficulty
        self.state = self.state_levels[self.current_difficulty]
        self.paint()

    def cycle_difficulty(self):
        if self.current_difficulty == 'easy':
            self.set_difficulty('medium')
        elif self.current_difficulty == 'medium':
            self.set_difficulty('hard')
        else:
            self.set_difficulty('easy')

    def sort_colors(self):
        self.state.sort()
        self.paint()

    def on_click(self, pos):
        if self.board.is_click_on_reset_button(pos):
            self.state.reset()
            self.show_initial_message = False
            self.first_click = False
        elif self.board.is_click_on_difficulty_button(pos):
            self.cycle_difficulty()
        elif self.board.is_click_on_sort_button(pos):
            self.sort_colors()
        elif self.board.is_click_on_palette(pos):
            self.selected_color = self.board.get_palette_color(pos, self.state.ncolors)
            self.first_click = True  # Set the flag after the first click
            self.show_initial_message = False
        elif self.first_click and not self.state.is_game_over() and \
                self.board.header_height <= pos[1] < self.board.board_height + self.board.header_height:
            index = self.board.get_clicked_cell(pos, int(self.state.n**0.5))
            self.state.click(index, self.selected_color)
            if self.state.score > self.highest_scores[self.current_difficulty]:
                self.highest_scores[self.current_difficulty] = self.state.score
        self.paint()

    def paint(self):
        if self.show_initial_message:
            message = self.initial_message
            self.board.screen.fill((0, 0, 0))
            self.board.draw_score(message)
            self.board.draw_palette(self.state, self.selected_color)
            pygame.display.flip()
        else:
            if self.state.is_game_over():
                message = f'Game Over! Your score is: {self.state.score}'
            else:
                message = f'Score: {self.state.score}'
            self.board.draw_all(self.state, message, self.selected_color, self.current_difficulty, self.highest_scores[self.current_difficulty])

    def start(self):
        self.paint()

    def run(self):
        self.start()
        running = True
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    self.on_click(event.pos)
        pygame.quit()

if __name__ == "__main__":
    game = Game()
    game.run()
